# Calculate CESM Indeces

This file just calculates all the indeces for all months for all cesmFullForcingFiles.

Could be generalised if desired.

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils.sstIndex as sst
import utils.pslIndex as psl


In [2]:
import xarray
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings('ignore')

Data for climatology

In [ ]:
sstClimatDs=fh.loadModelData('CESM-LME', 'SST', '0850cntl.001')
pslClimatDs=fh.loadModelData('CESM-LME', 'PSL', '0850cntl.001')

sstClimat=sst.calculateClimatology(sstClimatDs, 850, 2005)
pslClimat=psl.calculateClimatology(pslClimatDs, 850, 2005)

#loading both those things
for keys in sstClimat:
    sstClimat[keys].load()
    
pslClimat.load()

In [ ]:
sstClimatDs.close()
pslClimatDs.close()

For each experiment in Full Forcings, calculate all the indeces

In [ ]:
timePIndeces=dict()

for experiment in _model.cesmFullForcings:
    print(experiment)
    sstDs = fh.loadModelData('CESM-LME', 'SST', experiment)
    pslDs = fh.loadModelData('CESM-LME', 'PSL', experiment)
    pslIndex = psl.calculateSamIndex(pslDs, pslClimat)
    sstIndex = sst.calculateIndex(sstDs, sstClimat) #(reducing the model set at this step could save time?)
    indeces = xarray.merge([pslIndex, sstIndex])
    indeces.assign_attrs(climatology='calculated based on 850-2005 climatology from the control run')
    #indeces.to_netcdf('results/cesmMonthlyIndeces/' + experiment + '.nc')

    
    timePIndeces[experiment] = averageForTimePeriod(indeces)

# RCP8.5 extensions are a special case

    if experiment in ['002','003', '008', '009']:

        #pre2005Ds = xarray.load_dataset('results/cesmMonthlyIndeces/' + experiment + '.nc')

        print('LME.'+experiment)

        sstDs = fh.loadModelData('CESM-LME', 'SST', 'LME\.'+experiment)

        extensionIndex=sst.calculateIndex(sstDs, sstClimat)

        sstIndex = xarray.concat(
            #(pre2005Ds[[keys for keys in extensionIndex.data_vars]],
            (sstIndex,
             extensionIndex),
            'time'
        )


        pslDs = fh.loadModelData('CESM-LME', 'PSL', 'LME.'+experiment)

        extensionIndex=psl.calculateSamIndex(pslDs, pslClimat)

        pslIndex=xarray.concat(
            #(pre2005Ds[[keys for keys in extensionIndex.data_vars]],
            (pslIndex,
             extensionIndex),
            'time'
        )

        indeces = xarray.merge([pslIndex, sstIndex])
        indeces.assign_attrs(climatology='calculated based on 850-1850 climatology')
        #indeces.to_netcdf('results/cesmMonthlyIndeces/LME.' + experiment + '.nc')

    
    timePIndeces['LME.'+experiment] = averageForTimePeriod(indeces)

In [ ]:
#loading both those things
for keys in sstClimat:
    sstClimat[keys].close()
    
pslClimat.close()

# Calculate CESM Indeces

This grabs the monthly CESM indeces, and calculates the averages for the time periods of interest (aka the warm season), as set in the _indexDefitionions file

You need to run 'calculateCesmMonthlyIndeces.ipnyb' first

Theres a few examples of output at the bottom to sanity check this is meaningful

In [ ]:
#experimentSet = ['LME.002']
experimentSet = _model.cesmAll

The indeces for each month have already been calculated. So loading up the saves versions, and putting them into a dict of pandas arrays.

To make this all easier to deal with, convert the result into an xarray and add some extra relevant information as attributes.

This allows for filtering by index, year or experiment 

In [ ]:
# so i've just written the results into a dict of pandas array. This is kinda funky because I can't slice one index across multiple models
# instead, put the whole thing into an xarray with a dimension for the experiment
results=xarray.concat(
    [
        xarray.DataArray(timePIndeces[experiment], 
                         dims=['year','index'], 
                         coords={
                             'year':timePIndeces[experiment].index,
                             'index':timePIndeces[experiment].columns,
                             'experiment': experiment})
    for experiment in experimentSet 
    ],
    dim='experiment'
)

results.assign_attrs(indeces.attrs)
results.assign_attrs(_index.monthsOfInterest)



Save the result for use later

In [ ]:
results.to_netcdf('results/cesmTradIndeces2.nc')

# Some sanity checks to see if it looks ok:


In [ ]:
# plotSomething
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)

#results.sel(experiment='001', index='nino34').plot(label='001')
results.sel(experiment='LME.002', index='nino34NoDetrend').plot(label='LME.001')
results.sel(experiment='LME.002', index='nino34').plot(label='LME.001')

#Tidy up a bit
plt.title("CESM Full Forcing Nino34 Ensemble")
plt.ylim(-4,4)
plt.minorticks_on()
plt.legend()

In [ ]:
# A multi-experiment mean?

mean = results.sel(index='sam').mean(dim='experiment')
std = results.sel(index='sam').std(dim='experiment')

In [ ]:
plt.figure()
mean.plot()
std.plot()

In [ ]:
# why limit yourself to one index?

mean = results.sel(experiment=_model.cesmFullForcings).mean(dim='experiment')
std = results.sel(experiment=_model.cesmFullForcings).std(dim='experiment')

In [ ]:
std